<a href="https://colab.research.google.com/github/DineshGujjeti/Infosys_Springboard_CodeGenie/blob/main/Milestone4/code/milestone4_codeGenie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Install Dependencies (UPDATED)
print("📦 Installing all required packages...")

# Set up environment variables from Colab Secrets
import os
from google.colab import userdata

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['JWT_SECRET_KEY'] = userdata.get('JWT_SECRET_KEY')
os.environ['NGROK_TOKEN'] = userdata.get('NGROK_TOKEN')

# Install all required packages
!pip install --upgrade pip

# PyTorch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Transformers, Accelerate, and BitsAndBytes for 4-bit loading
!pip install transformers accelerate bitsandbytes

# Streamlit and utilities
# --- ADDED 'radon' TO THIS LIST ---
!pip install streamlit pyngrok PyJWT bcrypt pandas graphviz plotly-express radon

print("✅ All dependencies installed.")

📦 Installing all required packages...
Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [radon]
✅ All dependencies installed.


In [ ]:
%%writefile app.py

import streamlit as st
import sqlite3
import bcrypt
import jwt
import os
import graphviz
import ast
import torch
import pandas as pd
import plotly.express as px
import time
import gc # Garbage Collector
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datetime import datetime, timedelta

# --- 1. CONFIGURATION & SECRETS ---

HF_TOKEN = os.environ.get('HF_TOKEN')
JWT_SECRET = os.environ.get('JWT_SECRET_KEY', "dummy_jwt")
NGROK_TOKEN = os.environ.get('NGROK_TOKEN', "dummy_ngrok")

DB_NAME = "users.db"

# --- 2. DATABASE UTILITIES ---

def init_db():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL,
        security_question TEXT NOT NULL,
        security_answer_hash TEXT NOT NULL,
        role TEXT NOT NULL DEFAULT 'user'
    )
    ''')
    c.execute('''
    CREATE TABLE IF NOT EXISTS activity_log (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        timestamp TEXT NOT NULL,
        prompt TEXT NOT NULL,
        model_name TEXT NOT NULL,
        language TEXT NOT NULL,
        response_time REAL,
        complexity_score INTEGER DEFAULT 0,
        feedback_score INTEGER DEFAULT 0,
        FOREIGN KEY (user_id) REFERENCES users (id)
    )
    ''')
    c.execute("SELECT COUNT(*) FROM users")
    user_count = c.fetchone()[0]
    conn.commit()
    conn.close()
    return user_count == 0

def hash_text(text):
    salt = bcrypt.gensalt()
    return bcrypt.hashpw(text.encode('utf-8'), salt)

def check_hash(text, hashed):
    return bcrypt.checkpw(text.encode('utf-8'), hashed)

def add_user(username, password, question, answer, role='user'):
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        password_hash = hash_text(password)
        answer_hash = hash_text(answer)
        c.execute(
            "INSERT INTO users (username, password_hash, security_question, security_answer_hash, role) VALUES (?, ?, ?, ?, ?)",
            (username, password_hash, question, answer_hash, role)
        )
        conn.commit()
        return True
    except sqlite3.IntegrityError:
        return False
    finally:
        conn.close()

def verify_user(username, password):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT password_hash, role FROM users WHERE username=?", (username,))
    user = c.fetchone()
    conn.close()
    if user:
        password_hash, role = user
        if check_hash(password, password_hash):
            return {"username": username, "role": role}
    return None

def update_user_password(username, new_password):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    new_password_hash = hash_text(new_password)
    c.execute("UPDATE users SET password_hash = ? WHERE username = ?", (new_password_hash, username))
    conn.commit()
    conn.close()

def get_all_users():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT id, username, role FROM users")
    users = c.fetchall()
    conn.close()
    return users

def get_user_details(username):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT username, security_question, security_answer_hash FROM users WHERE username=?", (username,))
    user = c.fetchone()
    conn.close()
    if user:
        return {"username": user[0], "question": user[1], "answer_hash": user[2]}
    return None

# --- CODE QUALITY FUNCTION (STUBBED) ---
def calculate_complexity(code):
    return 0, 'N/A'

# --- LOGGING AND QUERY FUNCTIONS ---

def log_user_activity(username, prompt, model_name, language, response_time, complexity_score=0):
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        c.execute("SELECT id FROM users WHERE username=?", (username,))
        user_id = c.fetchone()
        if user_id:
            user_id = user_id[0]
        else:
            return
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        c.execute(
            "INSERT INTO activity_log (user_id, timestamp, prompt, model_name, language, response_time, complexity_score) VALUES (?, ?, ?, ?, ?, ?, ?)",
            (user_id, timestamp, prompt, model_name, language, response_time, complexity_score)
        )
        conn.commit()
        c.execute("SELECT last_insert_rowid()")
        st.session_state['last_activity_id'] = c.fetchone()[0]
    except Exception as e:
        print(f"Error logging activity to DB: {e}")
    finally:
        if conn:
            conn.close()

def update_feedback_score(activity_id, score):
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        c.execute("UPDATE activity_log SET feedback_score = ? WHERE id = ?", (score, activity_id))
        conn.commit()
        if 'last_activity_id' in st.session_state:
            del st.session_state['last_activity_id']
        st.toast(f"Feedback submitted! Thank you.")
    except Exception as e:
        st.error(f"Failed to submit feedback: {e}")
    finally:
        if conn:
            conn.close()

# --- DASHBOARD QUERY FUNCTIONS ---
def get_total_users():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT COUNT(id) FROM users")
    count = c.fetchone()[0]
    conn.close()
    return count

def get_total_queries():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT COUNT(id) FROM activity_log")
    count = c.fetchone()[0]
    conn.close()
    return count

def get_average_latency():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT AVG(response_time) FROM activity_log")
    avg_latency = c.fetchone()[0]
    conn.close()
    return f"{avg_latency:.1f}" if avg_latency is not None else "0.0"

def get_trending_queries():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("""
        SELECT prompt, COUNT(prompt) as count
        FROM activity_log
        GROUP BY prompt
        ORDER BY count DESC
        LIMIT 5
    """)
    data = c.fetchall()
    conn.close()
    trending_data = [
        {'Query': row[0][:50] + '...' if len(row[0]) > 50 else row[0], 'Count': row[1]}
        for row in data
    ]
    return pd.DataFrame(trending_data)

def get_language_usage():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("""
        SELECT language, COUNT(language) as count
        FROM activity_log
        GROUP BY language
    """)
    data = c.fetchall()
    conn.close()
    df = pd.DataFrame(data, columns=['Language', 'Count'])
    if df.empty:
        return pd.DataFrame({'Language': ['None'], 'Usage %': [100]})
    total_count = df['Count'].sum()
    df['Usage %'] = (df['Count'] / total_count) * 100
    df = df.sort_values(by='Usage %', ascending=False)
    return df[['Language', 'Usage %']]

def get_positive_feedback_rate():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT COUNT(id) FROM activity_log WHERE feedback_score != 0")
    total_feedback = c.fetchone()[0]
    if total_feedback == 0:
        conn.close()
        return "N/A", None
    c.execute("SELECT COUNT(id) FROM activity_log WHERE feedback_score > 0")
    positive_feedback = c.fetchone()[0]
    rate = (positive_feedback / total_feedback) * 100
    conn.close()
    return f"{rate:.0f}%", None

def get_average_complexity():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT AVG(complexity_score) FROM activity_log WHERE complexity_score > 0")
    avg_complexity = c.fetchone()[0]
    conn.close()
    return f"{avg_complexity:.1f}" if avg_complexity is not None else "N/A"

def get_user_history(username):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT id FROM users WHERE username=?", (username,))
    user_id = c.fetchone()[0]
    c.execute("""
        SELECT timestamp, model_name, language, prompt, response_time, complexity_score, feedback_score
        FROM activity_log
        WHERE user_id = ?
        ORDER BY timestamp DESC
        LIMIT 10
    """, (user_id,))
    data = c.fetchall()
    conn.close()
    history = pd.DataFrame(data, columns=['Timestamp', 'Model', 'Lang', 'Prompt', 'Latency (s)', 'CC Score', 'Feedback'])
    history['Latency (s)'] = history['Latency (s)'].round(2)
    history['Feedback'] = history['Feedback'].map({1: '👍', -1: '👎', 0: '—'})
    return history

def get_full_history():
    conn = sqlite3.connect(DB_NAME)
    history_df = pd.read_sql_query("SELECT * FROM activity_log ORDER BY timestamp DESC LIMIT 20", conn)
    conn.close()
    return history_df

def get_full_feedback():
    conn = sqlite3.connect(DB_NAME)
    history_df = pd.read_sql_query("SELECT * FROM activity_log", conn)
    feedback_df = pd.read_sql_query("SELECT * FROM activity_log WHERE feedback_score != 0", conn)
    conn.close()

    if not feedback_df.empty:
        feedback_with_history = pd.merge(feedback_df, history_df, left_on='id', right_on='id', how='left')
        return feedback_with_history[['timestamp_x', 'prompt_x', 'model_name_x', 'feedback_score_x']].tail(10)
    return pd.DataFrame()


# --- 3. AUTHENTICATION (JWT) ---

def create_jwt(username, role):
    payload = {
        'sub': username,
        'role': role,
        'iat': datetime.utcnow(),
        'exp': datetime.utcnow() + timedelta(hours=24)
    }
    token = jwt.encode(payload, JWT_SECRET, algorithm='HS256')
    return token

def verify_jwt(token):
    try:
        payload = jwt.decode(token, JWT_SECRET, algorithms=['HS256'])
        return payload
    except jwt.ExpiredSignatureError:
        st.error("Session expired. Please log in again.")
        return None
    except jwt.InvalidTokenError:
        st.error("Invalid token. Please log in again.")
        return None

# --- 4. DYNAMIC MODEL LOADING (MEMORY EFFICIENT) ---
# FIX 5: Replaced @st.cache_resource with dynamic "load-and-swap" logic

MODELS = {
    "Gemma-2B-IT": "google/gemma-2b-it",
    "DeepSeek-Coder-1.3B": "deepseek-ai/deepseek-coder-1.3b-instruct",
    "Phi-2": "microsoft/phi-2"
}

def load_model_from_hub(model_path):
    token = os.environ.get('HF_TOKEN')
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, token=token)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=quantization_config,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        token=token
    )
    return model, tokenizer

def get_model_and_tokenizer(model_name_key):
    model_path = MODELS.get(model_name_key)
    if not model_path:
        st.error(f"Model key {model_name_key} not found in MODELS dict.")
        return None, None

    if st.session_state.get('loaded_model_name') != model_path:
        if 'loaded_model' in st.session_state:
            print(f"Unloading old model: {st.session_state.loaded_model_name}")
            del st.session_state.loaded_model
            del st.session_state.loaded_tokenizer
            gc.collect()
            torch.cuda.empty_cache()
            print("Old model unloaded.")

        print(f"Loading new model: {model_path}...")
        with st.spinner(f"Loading {model_name_key}... (This may take a moment)"):
            try:
                model, tokenizer = load_model_from_hub(model_path)
            except Exception as e:
                st.error(f"Failed to load model {model_path}: {e}")
                if 'loaded_model_name' in st.session_state: del st.session_state.loaded_model_name
                return None, None

        st.session_state.loaded_model_name = model_path
        st.session_state.loaded_model = model
        st.session_state.loaded_tokenizer = tokenizer
        print("New model loaded.")

    else:
        print(f"Using cached model: {model_path}")

    return st.session_state.loaded_model, st.session_state.loaded_tokenizer

def generate_code(prompt, model_name_key):
    model, tokenizer = get_model_and_tokenizer(model_name_key)
    if model is None:
        return f"Error: Model {model_name_key} failed to load."

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    attention_mask = inputs.attention_mask
    output_ids = model.generate(
        inputs.input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    response_tokens = output_ids[0][inputs.input_ids.shape[-1]:]
    code = tokenizer.decode(response_tokens, skip_special_tokens=True)

    # FIX 4: Handle empty or invalid model responses
    if not code or not code.strip() or "undefined" in code:
        return "```\nError: Model returned an empty or invalid response. Please try again.\n```"

    formatted_code = f"```python\n{code.strip()}\n```"
    return formatted_code

def query_hf_model(payload, model_name_key):
    try:
        return generate_code(payload, model_name_key)
    except Exception as e:
        return f"Error: Code Generation/Explanation failed: {str(e)}"

# --- 5. AST VISUALIZER ---

class ASTVisualizer:
    def __init__(self):
        self.graph = graphviz.Digraph(comment='Python AST', graph_attr={'rankdir': 'TB', 'bgcolor': '#333333'}, node_attr={'style': 'filled', 'color': '#FF6347', 'fontcolor': 'white', 'fillcolor': '#444444'})

    def traverse(self, node, parent_id=None):
        node_id = str(id(node))
        node_label = type(node).__name__
        if hasattr(node, 'id') and isinstance(node.id, str):
            node_label = f"Name\n(id='{node.id}')"
        elif isinstance(node, ast.Constant):
            node_label = f"Constant\n(value={repr(node.value)})"
        elif isinstance(node, ast.Attribute):
            node_label = f"Attribute\n(attr='{node.attr}')"
        elif isinstance(node, ast.FunctionDef):
            node_label = f"FunctionDef\n(name='{node.name}')"
        self.graph.node(node_id, label=node_label)
        if parent_id:
            self.graph.edge(parent_id, node_id)
        for field, value in ast.iter_fields(node):
            if isinstance(value, list):
                for item in value:
                    if isinstance(item, ast.AST):
                        self.traverse(item, node_id)
            elif isinstance(value, ast.AST):
                self.traverse(value, node_id)

    def get_graph(self, code):
        try:
            tree = ast.parse(code)
            self.traverse(tree)
            return self.graph
        except SyntaxError as e:
            st.error(f"Python Syntax Error: {e}")
            return None
        except Exception as e:
            st.error(f"Error building AST: {e}")
            return None

# --- 6. STREAMLIT APP UI ---

def feedback_callback(score):
    if 'last_activity_id' in st.session_state:
        # This is a temporary score holder. The form will submit the final score.
        st.session_state.feedback_score = score

def initialize_state():
    # Base state
    if "logged_in" not in st.session_state: st.session_state.logged_in = False
    if "username" not in st.session_state: st.session_state.username = None
    if "role" not in st.session_state: st.session_state.role = None
    if "page" not in st.session_state: st.session_state.page = "Login"
    if "token" not in st.session_state: st.session_state.token = None
    if "chat_history" not in st.session_state: st.session_state.chat_history = []
    if "last_output" not in st.session_state: st.session_state.last_output = ""
    if "last_activity_id" not in st.session_state: st.session_state.last_activity_id = None
    if "reset_step" not in st.session_state: st.session_state.reset_step = 1
    if "reset_user" not in st.session_state: st.session_state.reset_user = None

    # DYNAMIC MODEL LOADING STATE
    if 'loaded_model_name' not in st.session_state: st.session_state.loaded_model_name = None
    if 'loaded_model' not in st.session_state: st.session_state.loaded_model = None
    if 'loaded_tokenizer' not in st.session_state: st.session_state.loaded_tokenizer = None

    # FIX 3: Explainer state
    if 'explanation' not in st.session_state: st.session_state.explanation = ""
    if 'ast_graph' not in st.session_state: st.session_state.ast_graph = None

def show_login_page(is_first_user):
    st.title("CodeGenie 🧞")
    col1, col2 = st.columns([2, 3])
    with col1:
        st.subheader("Welcome Back")
    with col2:
        with st.container(border=True):
            tab1, tab2 = st.tabs(["Login", "Sign Up"])
            with tab1:
                with st.form("login_form"):
                    username = st.text_input("Username", key="login_username")
                    password = st.text_input("Password", type="password", key="login_password")
                    login_button = st.form_submit_button("Login", use_container_width=True)
                    if login_button:
                        user_data = verify_user(username, password)
                        if user_data:
                            st.session_state.logged_in = True
                            st.session_state.username = user_data["username"]
                            st.session_state.role = user_data["role"]
                            st.session_state.token = create_jwt(user_data["username"], user_data["role"])
                            st.session_state.page = "Generator"
                            st.rerun()
                        else:
                            st.error("Invalid username or password")
                if st.button("Forgot Password?", type="primary"):
                    st.session_state.page = "Forgot Password"
                    st.rerun()
            with tab2:
                with st.form("signup_form"):
                    st.markdown("### Create Your Account")
                    if is_first_user:
                        st.info("✨ You will be the first user! You will be registered as an **Admin**.")
                    new_username = st.text_input("Username*")
                    # FIX 1: Corrected password syntax
                    new_password = st.text_input("Password*", type="password")
                    new_password_confirm = st.text_input("Confirm Password*", type="password")
                    st.markdown("---")
                    st.markdown("**Security Question (for password reset)**")
                    question = st.selectbox("Select a question*", ["What was the name of your first pet?", "What is your mother's maiden name?", "What city were you born in?", "What was your first car?"])
                    answer = st.text_input("Your Answer*", type="password") # Changed to text_input

                    signup_button = st.form_submit_button("Sign Up", use_container_width=True)
                    if signup_button:
                        if not (new_username and new_password and new_password_confirm and answer):
                            st.error("Please fill out all required fields.")
                        elif new_password != new_password_confirm:
                            st.error("Passwords do not match.")
                        else:
                            role = 'admin' if is_first_user else 'user'
                            success = add_user(new_username, new_password, question, answer, role)
                            if success:
                                st.success(f"User '{new_username}' created successfully as {'an Admin' if role == 'admin' else 'a User'}. Please log in.")
                            else:
                                st.error("Username already exists.")

def show_forgot_password_page():
    st.title("Reset Password")
    with st.container(border=True):
        if "reset_user" not in st.session_state: st.session_state.reset_user = None
        if "reset_step" not in st.session_state: st.session_state.reset_step = 1

        if st.session_state.reset_step == 1:
            with st.form("step1_username"):
                username = st.text_input("Enter your username")
                submit_user = st.form_submit_button("Next")
                if submit_user:
                    user_details = get_user_details(username)
                    if user_details:
                        st.session_state.reset_user = user_details
                        st.session_state.reset_step = 2
                        st.rerun()
                    else:
                        st.error("Username not found.")
        if st.session_state.reset_step == 2:
            st.info(f"**Security Question:** {st.session_state.reset_user['question']}")
            with st.form("step2_answer"):
                # FIX 1: Corrected password syntax
                answer = st.text_input("Your Answer", type="password")
                submit_answer = st.form_submit_button("Verify Answer")
                if submit_answer:
                    if check_hash(answer, st.session_state.reset_user['answer_hash']):
                        st.session_state.reset_step = 3
                        st.rerun()
                    else:
                        st.error("Incorrect answer.")
        if st.session_state.reset_step == 3:
            st.success("Verification successful!")
            with st.form("step3_new_password"):
                new_password = st.text_input("New Password", type="password")
                confirm_new_password = st.text_input("Confirm New Password", type="password")
                submit_pass = st.form_submit_button("Reset Password")
                if submit_pass:
                    if not new_password or new_password != confirm_new_password:
                        st.error("Passwords do not match or are empty.")
                    else:
                        update_user_password(st.session_state.reset_user['username'], new_password)
                        st.success("Password reset successfully! You can now log in.")
                        st.session_state.reset_step = 1
                        st.session_state.reset_user = None
                        st.session_state.page = "Login"
                        st.rerun()
    if st.button("← Back to Login"):
        st.session_state.page = "Login"
        st.session_state.reset_step = 1
        st.session_state.reset_user = None
        st.rerun()

def preload_selected_model():
    if st.session_state.gen_model_key:
        print(f"Pre-loading model: {st.session_state.gen_model_key}")
        get_model_and_tokenizer(st.session_state.gen_model_key)

def show_generator_page():
    st.title("🤖 Code Generator")
    model_keys = list(MODELS.keys())
    selected_model = st.selectbox(
        "Select AI Model",
        model_keys,
        key='gen_model_key',
        on_change=preload_selected_model
    )

    if 'loaded_model' not in st.session_state or st.session_state.loaded_model is None:
        preload_selected_model()

    chat_container = st.container(height=400, border=True)
    with chat_container:
        for entry in st.session_state.chat_history:
            with st.chat_message(entry["role"]):
                st.markdown(entry["content"])
    prompt = st.text_area("Your prompt:", key="prompt_input", height=100)
    col1, col2, col3 = st.columns([2, 1, 1])

    with col1:
        if st.button("Generate Code 🚀", use_container_width=True, type="primary"):
            if prompt:
                st.session_state.chat_history.append({"role": "user", "content": prompt})
                with chat_container:
                    with st.chat_message("user"):
                        st.markdown(prompt)
                    with st.chat_message("assistant"):
                         message_placeholder = st.empty()

                with st.spinner(f"CodeGenie is thinking with {selected_model}..."):
                    start_time = time.time()
                    try:
                        response = generate_code(prompt, selected_model)
                        complexity_score = 0
                    except Exception as e:
                        response = f"Error: {str(e)}"
                        complexity_score = 0
                    end_time = time.time()
                    latency = end_time - start_time

                log_user_activity(st.session_state.username, prompt, selected_model, "Python", latency, complexity_score)
                message_placeholder.markdown(response)
                st.session_state.chat_history.append({"role": "assistant", "content": response})
                if not response.startswith("Error:"):
                    st.session_state.last_output = response
                st.rerun()
            else:
                st.warning("Please enter a prompt.")

    with col2:
        if st.button("📋 Copy Last Output", use_container_width=True):
            if st.session_state.last_output:
                st.components.v1.html(f"<script>navigator.clipboard.writeText({repr(st.session_state.last_output)});</script>", height=0)
                st.toast("Output copied to clipboard!")
            else:
                st.toast("No output to copy yet.")
    with col3:
        if st.button("🗑️ Clear History", use_container_width=True):
            st.session_state.chat_history = []
            st.session_state.last_output = ""
            st.session_state.explanation = ""
            st.session_state.ast_graph = None
            st.rerun()

    # FIX 2: Replaced buttons with star/comment feedback form
    if 'last_activity_id' in st.session_state and st.session_state.last_activity_id is not None:
        st.markdown("---")
        with st.form(key="feedback_form_gen"):
            st.subheader("Was this response helpful?")
            rating = st.radio(
                "Rate this response:",
                options=[1, 2, 3, 4, 5],
                format_func=lambda x: "⭐" * x,
                horizontal=True
            )
            comment = st.text_area("Additional feedback (optional):")

            if st.form_submit_button("Submit Feedback"):
                try:
                    # Use the rating as the score
                    update_feedback_score(st.session_state.last_activity_id, rating)
                    st.success("Thank you for your feedback!")
                    st.session_state.last_activity_id = None # Clear after submitting
                    st.rerun()
                except Exception as e:
                    st.error(f"Failed to submit feedback: {e}")

def preload_explainer_model():
    if st.session_state.exp_model_key:
        print(f"Pre-loading explainer model: {st.session_state.exp_model_key}")
        get_model_and_tokenizer(st.session_state.exp_model_key)

def show_explainer_page():
    st.title("🔬 Code Explainer & AST Visualizer")
    lang = st.selectbox("Select Language", ["Python", "JavaScript", "SQL"])
    code = st.text_area("Paste your code here:", height=250)
    model_keys = list(MODELS.keys())
    explainer_model_name = st.selectbox(
        "Select Explainer Model",
        model_keys,
        index=0,
        key='exp_model_key',
        on_change=preload_explainer_model
    )

    if 'loaded_model' not in st.session_state or st.session_state.loaded_model is None:
        preload_explainer_model()

    if st.button("Explain & Visualize", type="primary"):
        if not code:
            st.warning("Please paste some code to explain.")
            st.session_state.explanation = "" # Clear old explanations
            st.session_state.ast_graph = None
            return

        # Clear old results before generating new ones
        st.session_state.explanation = ""
        st.session_state.ast_graph = None

        with st.spinner(f"Generating explanation with {explainer_model_name}..."):
            explain_prompt = f"Explain the following {lang} code. Describe what it does, step by step. Provide the explanation in markdown format, using code blocks for the code.\n\n```\n{code}\n```"
            start_time = time.time()
            try:
                explanation = query_hf_model(explain_prompt, explainer_model_name)
                complexity_score = 0
            except Exception as e:
                explanation = f"Error: {str(e)}"
                complexity_score = 0
            end_time = time.time()
            latency = end_time - start_time
            log_user_activity(st.session_state.username, code, explainer_model_name, lang, latency, complexity_score)

            # FIX 3: Persist explanation to session state
            st.session_state.explanation = explanation

        if lang == "Python":
            visualizer = ASTVisualizer()
            graph = visualizer.get_graph(code)
            if graph:
                # FIX 3: Persist graph to session state
                st.session_state.ast_graph = graph

        st.session_state.last_output = "Explainer Run" # For feedback
        # DO NOT RERUN. Let the results display.

    # FIX 3: Display persistent results *outside* the button click logic
    if st.session_state.explanation:
        col1, col2 = st.columns(2)
        with col1:
            st.subheader("AI Explanation")
            st.markdown(st.session_state.explanation)
        with col2:
            st.subheader("Abstract Syntax Tree (AST)")
            if st.session_state.ast_graph:
                st.graphviz_chart(st.session_state.ast_graph)
            elif lang == "Python":
                 st.error("Could not generate AST for this Python code.")
            else:
                st.info(f"AST visualization is only available for Python.")

    # FIX 2: Replaced buttons with star/comment feedback form
    if 'last_activity_id' in st.session_state and st.session_state.last_activity_id is not None and st.session_state.last_output == "Explainer Run":
        st.markdown("---")
        with st.form(key="feedback_form_exp"):
            st.subheader("Was this explanation helpful?")
            rating = st.radio(
                "Rate this response:",
                options=[1, 2, 3, 4, 5],
                format_func=lambda x: "⭐" * x,
                horizontal=True
            )
            comment = st.text_area("Additional feedback (optional):")

            if st.form_submit_button("Submit Feedback"):
                try:
                    update_feedback_score(st.session_state.last_activity_id, rating)
                    st.success("Thank you for your feedback!")
                    st.session_state.last_activity_id = None
                    st.rerun()
                except Exception as e:
                    st.error(f"Failed to submit feedback: {e}")

def show_profile_page():
    st.title(f"Profile: {st.session_state.username}")
    st.info(f"**Username:** `{st.session_state.username}`\n\n**Role:** `{st.session_state.role}`")
    st.subheader("Change Password")
    with st.form("change_password_form"):
        current_password = st.text_input("Current Password", type="password")
        new_password = st.text_input("New Password", type="password")
        confirm_new_password = st.text_input("Confirm New Password", type="password")
        submit_change = st.form_submit_button("Update Password")
        if submit_change:
            user_data = verify_user(st.session_state.username, current_password)
            if not user_data:
                st.error("Incorrect current password.")
            elif not new_password or new_password != confirm_new_password:
                st.error("New passwords do not match or are empty.")
            else:
                update_user_password(st.session_state.username, new_password)
                st.success("Password updated successfully!")

    st.markdown("---")
    st.subheader("Recent Activity History")
    history_df = get_user_history(st.session_state.username)
    if not history_df.empty:
        st.dataframe(history_df, use_container_width=True, hide_index=True)
    else:
        st.info("No activity logged yet.")

# --- FIX 6: Added back the advanced Admin Dashboard ---
def show_admin_dashboard_page():
    if st.session_state.role != 'admin':
        st.error("You do not have permission to view this page.")
        return
    st.title("👑 Admin Analytics Dashboard")
    total_users = get_total_users()
    total_queries = get_total_queries()
    avg_latency = get_average_latency()
    trending_queries_df = get_trending_queries()
    lang_usage_df = get_language_usage()
    feedback_rate, feedback_delta = get_positive_feedback_rate()
    avg_complexity = get_average_complexity()
    st.subheader("Key Usage Statistics")
    col1, col2, col3, col4 = st.columns(4)
    with col1:
        with st.container(border=True):
            st.metric(label="Total Users", value=f"{total_users}", delta=None)
    with col2:
        with st.container(border=True):
            st.metric(label="Total Queries", value=f"{total_queries}", delta=None)
    with col3:
        with st.container(border=True):
            st.metric(label="Avg. Response Time", value=f"{avg_latency}s", delta=None)
    with col4:
        with st.container(border=True):
            st.metric(label="Positive Feedback Rate", value=feedback_rate, delta=feedback_delta)
    st.markdown("---")
    colA, colB = st.columns([3, 2])
    with colA:
        st.subheader("Trending Queries (Top 5)")
        st.dataframe(trending_queries_df, hide_index=True, use_container_width=True)
    with colB:
        st.subheader("Language Usage Breakdown")
        if not lang_usage_df.empty and lang_usage_df['Usage %'].sum() > 0:
            fig = px.pie(lang_usage_df, names='Language', values='Usage %', title='Language Usage')
            st.plotly_chart(fig, use_container_width=True)
        else:
             st.info("No language usage data recorded yet.")
    st.markdown("---")
    st.subheader("Code Quality & Evaluation Metrics")
    colC, colD = st.columns(2)
    with colC:
         with st.container(border=True):
             st.metric(label="Avg. Cyclomatic Complexity (CC)", value=avg_complexity, delta=None)
    with colD:
         st.info("CC scores are N/A in this version for performance.")


def show_admin_users_page():
    if st.session_state.role != 'admin':
        st.error("You do not have permission to view this page.")
        return
    st.title("👑 Admin: User Management")

    tab_users, tab_history, tab_feedback = st.tabs(["User List", "Full Query History", "Full Feedback Log"])

    with tab_users:
        st.subheader("All Registered Users")
        users = get_all_users()
        if users:
            df_data = [{"ID": user[0], "Username": user[1], "Role": user[2]} for user in users]
            st.dataframe(pd.DataFrame(df_data), use_container_width=True, hide_index=True)
        else:
            st.info("No users found (except you).")

    with tab_history:
        st.subheader("Full Query History (Last 20)")
        st.dataframe(get_full_history(), use_container_width=True)

    with tab_feedback:
        st.subheader("Full Feedback Log (Last 10)")
        st.dataframe(get_full_feedback(), use_container_width=True)


# --- 7. MAIN APP ROUTER ---

def main():
    st.set_page_config(page_title="CodeGenie", page_icon="🧞", layout="wide")
    is_first_user = init_db()
    initialize_state()

    if st.session_state.token:
        payload = verify_jwt(st.session_state.token)
        if payload:
            st.session_state.logged_in = True
            st.session_state.username = payload['sub']
            st.session_state.role = payload['role']
        else:
            for key in list(st.session_state.keys()):
                del st.session_state[key]
            st.rerun()

    if not st.session_state.logged_in:
        if st.session_state.page == "Forgot Password":
            show_forgot_password_page()
        else:
            show_login_page(is_first_user)
    else:
        with st.sidebar:
            st.title(f"CodeGenie 🧞")
            st.markdown(f"Welcome, **{st.session_state.username}**!")
            st.markdown("---")
            def set_page(page_name):
                st.session_state.page = page_name

            st.button("Code Generator", use_container_width=True, type="primary" if st.session_state.page == "Generator" else "secondary", on_click=set_page, args=("Generator",))
            st.button("Code Explainer", use_container_width=True, type="primary" if st.session_state.page == "Explainer" else "secondary", on_click=set_page, args=("Explainer",))
            st.button("My Profile", use_container_width=True, type="primary" if st.session_state.page == "Profile" else "secondary", on_click=set_page, args=("Profile",))

            if st.session_state.role == 'admin':
                st.markdown("---")
                st.markdown("*Admin Tools*")
                # FIX 6: Replaced "Admin Panel" with the two new pages
                st.button("Admin Dashboard", use_container_width=True, type="primary" if st.session_state.page == "Dashboard" else "secondary", on_click=set_page, args=("Dashboard",))
                st.button("Manage Users", use_container_width=True, type="primary" if st.session_state.page == "Admin" else "secondary", on_click=set_page, args=("Admin",))

            st.markdown("---")
            if st.button("Logout 🔒", use_container_width=True):
                for key in list(st.session_state.keys()):
                    del st.session_state[key]
                st.rerun()

        if st.session_state.page == "Generator":
            show_generator_page()
        elif st.session_state.page == "Explainer":
            show_explainer_page()
        elif st.session_state.page == "Profile":
            show_profile_page()
        # FIX 6: Added routes for the new admin pages
        elif st.session_state.page == "Dashboard":
             show_admin_dashboard_page()
        elif st.session_state.page == "Admin":
             show_admin_users_page()
        else:
            st.session_state.page = "Generator"
            st.rerun()

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
# Cell 3: Launch the App
import os
import time
import subprocess
import socket
from google.colab import userdata
from pyngrok import ngrok

# ---- Clean up any old ngrok/Streamlit processes ----
print("🧹 Cleaning up old ngrok and Streamlit processes...")
!pkill -f ngrok 2>/dev/null || echo "No old ngrok"
!kill -9 $(lsof -t -i:8501) 2>/dev/null || echo "No old Streamlit"

# ---- Authenticate ngrok ----
ngrok.set_auth_token(os.environ['NGROK_TOKEN'])

# ---- Helper to wait for Streamlit ----
def wait_for_port(host="localhost", port=8501, timeout=90):
    start = time.time()
    while time.time() - start < timeout:
        try:
            with socket.create_connection((host, port), timeout=2):
                return True
        except OSError:
            time.sleep(1)
    return False

# ---- Start Streamlit ----
print("🚀 Starting Streamlit on port 8501...")
# Start the process and pipe logs to files
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=open("streamlit.log", "w"),
    stderr=open("streamlit.err", "w")
)

# ---- Wait until Streamlit is ready ----
print("⏳ Waiting for Streamlit to start...")
if not wait_for_port():
    print("❌ Streamlit did not start. Check app logs.")
    # Print the error logs if it fails
    !echo "--- STDOUT ---"
    !cat streamlit.log
    !echo "--- STDERR ---"
    !cat streamlit.err
else:
    # ---- Open ngrok tunnel only after Streamlit is ready ----
    print("🌐 Opening ngrok tunnel...")
    tunnel = ngrok.connect(8501)
    print(f"\n✅ Your Streamlit app is live!\n👉 {tunnel.public_url}\n")

🧹 Cleaning up old ngrok and Streamlit processes...
^C
🚀 Starting Streamlit on port 8501...
⏳ Waiting for Streamlit to start...
🌐 Opening ngrok tunnel...

✅ Your Streamlit app is live!
👉 https://rosemary-tricuspidate-earlene.ngrok-free.dev

